In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git
!pip install librosa==0.9.0 pydub noisereduce scikit-learn tensorflow torchaudio seaborn matplotlib transformers
!sudo apt update && sudo apt install ffmpeg

In [ ]:
arquivo = "/content/Gravação233.m4a"
!whisper {arquivo} --model medium --task transcribe --language pt --output_format txt

100%|█████████████████████████████████████| 1.42G/1.42G [00:31<00:00, 48.5MiB/s]
[00:00.000 --> 00:11.000]  Pessoal, hoje é dia da patrulha do consumidor que mostra uma blitz do Procon e da polícia civil no supermercado denunciado pelos clientes por falta de higiene.
[00:11.000 --> 00:19.000]  Durante a fiscalização foram encontrados alimentos vencidos, impróprios para consumo e armazenados de maneira inadequada.
[00:19.000 --> 00:21.000]  Acompanhe com atenção a reportagem.


In [ ]:
import os
import zipfile
import torch
import librosa
import torchaudio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from transformers import pipeline
import logging


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
class ProcessadorAudio:
    def __init__(self, taxa_amostragem_alvo=16000, n_fft=1024, hop_length=512, n_mels=64, max_pad_len=400):
        self.taxa_amostragem_alvo = taxa_amostragem_alvo
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.max_pad_len = max_pad_len

    def carregar_audio(self, caminho):
        waveform, sample_rate = torchaudio.load(caminho)
        if sample_rate != self.taxa_amostragem_alvo:
            resample_transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.taxa_amostragem_alvo)
            waveform = resample_transform(waveform)
        return waveform

    def extrair_caracteristicas(self, waveform):
        mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.taxa_amostragem_alvo,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            n_mels=self.n_mels
        )
        mel_spectrogram = mel_spectrogram_transform(waveform)
        mel_spectrogram = mel_spectrogram.mean(dim=0)
        pad = self.max_pad_len - mel_spectrogram.shape[-1]
        if pad > 0:
            mel_spectrogram = torch.nn.functional.pad(mel_spectrogram, (0, pad))
        else:
            mel_spectrogram = mel_spectrogram[:, :self.max_pad_len]
        return mel_spectrogram.numpy().flatten()

In [ ]:
class DetectorEmocao:
    def __init__(self, modelo):
        self.modelo = modelo

    def prever(self, caracteristicas):
        caracteristicas = caracteristicas[np.newaxis, ..., np.newaxis]
        previsoes = self.modelo.predict(caracteristicas)
        return np.argmax(previsoes, axis=1)[0]

In [ ]:
class ProcessadorNLP:
    def __init__(self):
        self.analisador_sentimento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

    def analisar_sentimento(self, texto):
        resultado = self.analisador_sentimento(texto)
        sentimento = resultado[0]['label']  # mapeando as saídas do modelo de NLP para as emotions esperadas

        mapeamento_sentimentos = {
            "1 star": "sad",
            "2 stars": "neutral",
            "3 stars": "neutral",
            "4 stars": "happy",
            "5 stars": "happy"
        }
        return mapeamento_sentimentos.get(sentimento, "neutral")

In [ ]:
class GeradorResposta:
    def __init__(self):
        self.respostas = {
            "happy": ["Que bom te ver feliz!", "Continue sorrindo!"],
            "sad": ["Lamento ouvir isso.", "Estou aqui para ajudar."],
            "angry": ["Por favor, respire fundo.", "Vamos tentar se acalmar."],
            "neutral": ["Como posso te ajudar hoje?", "O que gostaria de fazer?"]
        }

    def gerar_resposta(self, emocao_audio, emocao_texto): # Combina a emoção detectada no audio e texto para gerar uma resposta mais contextualizada
        if emocao_texto == "neutral":
            return np.random.choice(self.respostas[emocao_audio])
        return np.random.choice(self.respostas[emocao_texto])


In [ ]:
class AssistenteAudioEmocional:
    def __init__(self, processador_audio, detector_emocao, processador_nlp, gerador_resposta):
        self.processador_audio = processador_audio
        self.detector_emocao = detector_emocao
        self.processador_nlp = processador_nlp
        self.gerador_resposta = gerador_resposta
        self.buffer = []

    def processar_audio(self, caminho_audio):
        logging.info("Processando áudio...")
        waveform = self.processador_audio.carregar_audio(caminho_audio)
        caracteristicas = self.processador_audio.extrair_caracteristicas(waveform)
        emocao = self.detector_emocao.prever(caracteristicas)
        return emocao

    def transcrever_audio(self, caminho_audio):
        logging.info("Transcrevendo áudio...")
        !whisper {caminho_audio} --model medium --task transcribe --language pt --output_format txt
        with open(caminho_audio.replace('.m4a', '.txt'), 'r') as file:
            transcricao = file.read()
        return transcricao

    def processar_texto(self, texto):
        logging.info("Processando texto...")
        sentimento = self.processador_nlp.analisar_sentimento(texto)
        return sentimento

    def gerar_resposta(self, emocao_audio, emocao_texto):
        logging.info("Gerando resposta...")
        return self.gerador_resposta.gerar_resposta(emocao_audio, emocao_texto)

    def lidar_interacao(self, caminho_audio):
        emocao_audio = self.processar_audio(caminho_audio)
        transcricao = self.transcrever_audio(caminho_audio)
        emocao_texto = self.processar_texto(transcricao)
        resposta = self.gerar_resposta(emocao_audio, emocao_texto)
        return resposta

In [ ]:
def construir_modelo_emocao(input_shape, num_classes, caminho_pesos):
    modelo = Sequential([
        Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.2),
        Conv1D(128, 3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    if os.path.exists(caminho_pesos):   # logit para verificar se o arquivo de pesos existe
        modelo.load_weights(caminho_pesos)
        logging.info(f"Pesos carregados de {caminho_pesos}")
    else:
        logging.error(f"Arquivo de pesos não encontrado: {caminho_pesos}")
    return modelo

In [ ]:
!gdown 1p1S5yO1F77aegNeNe2UpVkN6nZicvm8D

Downloading...
From (original): https://drive.google.com/uc?id=1p1S5yO1F77aegNeNe2UpVkN6nZicvm8D
From (redirected): https://drive.google.com/uc?id=1p1S5yO1F77aegNeNe2UpVkN6nZicvm8D&confirm=t&uuid=efb17398-8251-4ee3-b4bc-ac97d7650c16
To: /content/best_model_emoUERJ.h5
100% 2.52G/2.52G [00:36<00:00, 68.1MB/s]


In [ ]:
model_path = '/content/best_model_emoUERJ.h5'
model = load_model(model_path)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 25600, 64)         256       
                                                                 
 batch_normalization_4 (Bat  (None, 25600, 64)         256       
 chNormalization)                                                
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 12800, 64)         0         
 g1D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 12800, 64)         0         
                                                                 
 conv1d_5 (Conv1D)           (None, 12800, 128)        24704     
                                                                 
 batch_normalization_5 (Bat  (None, 12800, 128)       

In [ ]:
input_shape = (400 * 64, 1)
num_classes = 4  # numero de classes
caminho_pesos = '/content/best_model_emoUERJ.h5'
modelo_emocao = construir_modelo_emocao(input_shape, num_classes, caminho_pesos)

In [ ]:
processador_audio = ProcessadorAudio()
detector_emocao = DetectorEmocao(modelo_emocao)
processador_nlp = ProcessadorNLP()
gerador_resposta = GeradorResposta()

assistente = AssistenteAudioEmocional(processador_audio, detector_emocao, processador_nlp, gerador_resposta)

In [ ]:
caminho_audio = "/content/Gravação233.m4a"
resposta = assistente.lidar_interacao(caminho_audio)
print("Resposta do Assistente:", resposta)

1/1 [==============================] - 0s 47ms/step
[00:00.000 --> 00:11.000]  Pessoal, hoje é dia da patrulha do consumidor que mostra uma blitz do Procon e da polícia civil no supermercado denunciado pelos clientes por falta de higiene.
[00:11.000 --> 00:19.000]  Durante a fiscalização foram encontrados alimentos vencidos, impróprios para consumo e armazenados de maneira inadequada.
[00:19.000 --> 00:21.000]  Acompanhe com atenção a reportagem.
Resposta do Assistente: Lamento ouvir isso.
